![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Prompt Notebook with Chat - Prompt Lab Notebook v1.1.0
This notebook contains steps and code to demonstrate inferencing of prompts
generated in Prompt Lab in watsonx.ai with a chat format. It introduces Python API commands
for authentication using API key and prompt inferencing using WML API.

**Note:** Notebook code generated using Prompt Lab will execute successfully.
If code is modified or reordered, there is no guarantee it will successfully execute.
For details, see: <a href="/docs/content/wsj/analyze-data/fm-prompt-save.html?context=wx" target="_blank">Saving your work in Prompt Lab as a notebook.</a>

Some familiarity with Python is helpful. This notebook uses Python 3.10.

## Notebook goals
The learning goals of this notebook are:

* Defining a Python function for obtaining credentials from the IBM Cloud personal API key
* Defining parameters of the Model object
* Using the Model object to generate response using the defined model id, parameters and the prompt input

# Setup

In [4]:
!pip install ibm_watsonx_ai
!pip install git+https://github.com/ibm-granite-community/utils

  Cloning https://github.com/ibm-granite-community/utils to /tmp/pip-req-build-k9zxne_1
  Running command git clone --filter=blob:none --quiet https://github.com/ibm-granite-community/utils /tmp/pip-req-build-k9zxne_1
  Resolved https://github.com/ibm-granite-community/utils to commit 3c725e243561d17b575a0a5496bfa3d3f28f6481
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for ibm-granite-community-utils: filename=ibm_granite_community_utils-0.1.dev112-py3-none-any.whl size=16277 sha256=8cb34be61af464d9ed30106e071577d986158808cae35d6995fe9b499783b2c2
  Stored in directory: /tmp/pip-ephem-wheel-cache-l_ehd7rm/wheels/e2/74/0e/e7dc80cad1c61a0c57be9aff96c6c6bbb058052bc5b9cac0ff
Successfully built ibm-granite-community-utils


## watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud personal API key. For details, see
<a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank">documentation</a>.


In [5]:
import os
from ibm_watsonx_ai import APIClient, Credentials
from ibm_granite_community.notebook_utils import get_env_var
import getpass

credentials = Credentials(
    url="https://us-south.ml.cloud.ibm.com",
    api_key=get_env_var("IBM_CLOUD_TOKEN")
)



IBM_CLOUD_TOKEN loaded from Google Colab secret.


# Inferencing
This cell demonstrated how we can use the model object as well as the created access token
to pair it with parameters and input string to obtain
the response from the the selected foundation model.

## Defining the model id
We need to specify model id that will be used for inferencing:


In [6]:
model_id = "mistralai/mistral-medium-2505"


## Defining the model parameters
We need to provide a set of model parameters that will influence the
result:

In [7]:
parameters = {
    "frequency_penalty": 0,
    "max_tokens": 2000,
    "presence_penalty": 0,
    "temperature": 0,
    "top_p": 1
}

## Defining the project id or space id
The API requires project id or space id that provides the context for the call. We will obtain
the id from the project or space in which this notebook runs:

In [10]:
project_id = get_env_var("PROJECT_ID")
space_id = get_env_var("SPACE_ID")


PROJECT_ID loaded from Google Colab secret.
SPACE_ID loaded from Google Colab secret.


## Defining the Model object
We need to define the Model object using the properties we defined so far:


In [11]:
from ibm_watsonx_ai.foundation_models import ModelInference

model = ModelInference(
	model_id = model_id,
	params = parameters,
	credentials = credentials,
	project_id = project_id,
	space_id = space_id
	)


## Defining the vector index
Initialize the vector index to query when chatting with the model.

In [12]:
from ibm_watsonx_ai.foundation_models.utils import Toolkit

vector_index_id = "e0cb43c3-093c-4481-8c97-cc3b0d6c076a"

def proximity_search( query ):

    api_client = APIClient(
        project_id=project_id,
        credentials=credentials,
    )

    document_search_tool = Toolkit(
        api_client=api_client
    ).get_tool("RAGQuery")

    config = {
    "vectorIndexId": vector_index_id,
    "projectId": project_id
    }

    results = document_search_tool.run(
        input=query,
        config=config
    )

    return results.get("output")

## Defining the inferencing input for chat
Foundation models supporting chat accept a system prompt that instructs the model on how to conduct the dialog. They also accept previous questions and answers to give additional context when inferencing. Each model has it's own string format for constructing the input.

Let us provide the input we got from the Prompt Lab and format it for the selected model:


In [13]:
chat_messages = [];


## Execution
Let us now use the defined Model object, pair it with the input, and generate the response to your question:


In [14]:
question = input("Question: ")
grounding = proximity_search(question)
chat_messages.append({
    "role": f"system",
    "content": f"""{grounding}

You are Mixtral Chat, an AI language model developed by Mistral AI. You are a cautious assistant. You carefully follow instructions. You are helpful and harmless and you follow ethical guidelines and promote positive behavior. You are a AI language model designed to function as a specialized Retrieval Augmented Generation (RAG) assistant. When generating responses, prioritize correctness, i.e., ensure that your response is correct given the context and user query, and that it is grounded in the context. Furthermore, make sure that the response is supported by the given document or context. When the question cannot be answered using the context or document, output the following response: "I cannot answer that question based on the provided document." Always make sure that your response is relevant to the question. If an explanation is needed, first provide the explanation or reasoning, and then give the final answer.

"""
})
chat_messages.append({"role": "user", "content": question})
generated_response = model.chat(messages=chat_messages)
print(generated_response)


Question: Based on my transactional data, can you give me any recommendations on how to better budget and lower my expenses? I want to save more, so I need to cut down on unnecessary spending.
{'id': 'chatcmpl-6a3844d4504cba2f1f63415cf21194ec---df8d809b-5c61-42df-8025-83d88dd64b93', 'object': 'chat.completion', 'model_id': 'mistralai/mistral-medium-2505', 'model': 'mistralai/mistral-medium-2505', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'Based on your transactional data, here are some recommendations to help you better budget and lower your expenses:\n\n1. **Identify High-Expense Categories**:\n   - **Rent**: This is a significant expense. Consider options like finding a more affordable place to live, getting a roommate, or negotiating your rent.\n   - **Food & Drink**: This category also shows high expenses. Plan your meals, cook at home, and reduce eating out.\n   - **Health & Fitness**: While important, look for more cost-effective alternatives like home 

In [15]:
output = generated_response['choices'][0]['message']['content']

In [16]:
from IPython.display import Markdown, display

def print_markdown(string):
  display(Markdown(string))

print_markdown(output)

Based on your transactional data, here are some recommendations to help you better budget and lower your expenses:

1. **Identify High-Expense Categories**:
   - **Rent**: This is a significant expense. Consider options like finding a more affordable place to live, getting a roommate, or negotiating your rent.
   - **Food & Drink**: This category also shows high expenses. Plan your meals, cook at home, and reduce eating out.
   - **Health & Fitness**: While important, look for more cost-effective alternatives like home workouts or community fitness programs.
   - **Travel**: This is another area with substantial spending. Plan trips in advance, look for deals, and consider more budget-friendly travel options.
   - **Entertainment**: This category can often be reduced by finding free or low-cost activities, such as community events or outdoor activities.

2. **Track and Categorize Expenses**:
   - Use budgeting tools or apps to track your expenses. Categorize them to see where your money is going each month.

3. **Set Budget Limits**:
   - Based on your income and necessary expenses, set monthly limits for each category. Stick to these limits to avoid overspending.

4. **Reduce Discretionary Spending**:
   - Look at categories like Shopping, Entertainment, and Travel. These are areas where you can often cut back without significantly impacting your quality of life.

5. **Increase Income**:
   - While not directly related to expenses, increasing your income through side jobs, investments, or other means can help you save more.

6. **Review and Adjust Regularly**:
   - Regularly review your budget and expenses. Adjust your spending habits as needed to stay on track with your savings goals.

7. **Emergency Fund**:
   - Ensure you have an emergency fund to cover unexpected expenses. This can help you avoid dipping into your savings or going into debt.

By following these recommendations, you can better manage your expenses and increase your savings.

# Next steps
You successfully completed this notebook! You learned how to use
watsonx.ai inferencing SDK to generate response from the foundation model
based on the provided input, model id and model parameters. Check out the
official watsonx.ai site for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2023 IBM. This notebook and its source code are released under the terms of the ILAN License.
Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for watsonx.ai Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for watsonx.ai Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="https://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF" target="_blank">License Terms</a>  